This file helps to summarize the output files over several runs.
It plots the train and prediction datasets of multiple runs in one large pdf file.

In [4]:
# script to run deepmod for ODEs

# General imports
import numpy as np
import matplotlib.pylab as plt
import os
import pandas as pd
import re

In [2]:
# load folder directory of batch
batch = "batch_10-02_miaSimS4"
out_dir = f"C:/Users/Maria/Documents/Masterstudium/Masterarbeit/deepmod_output/output_LRZ_Cluster/{batch}/"

# run_specifications
n_taxa = 4
# hl_number = 20
# hl_size = 60
# th = "0_001"

# list files in batch run
os.listdir(out_dir)

['output_batch_10-02_miaSimS4_run01_10_100_th0_001',
 'output_batch_10-02_miaSimS4_run01_10_40_th0_001',
 'output_batch_10-02_miaSimS4_run01_10_60_th0_001',
 'output_batch_10-02_miaSimS4_run01_15_100_th0_001',
 'output_batch_10-02_miaSimS4_run01_15_40_th0_001',
 'output_batch_10-02_miaSimS4_run01_15_60_th0_001',
 'output_batch_10-02_miaSimS4_run01_5_100_th0_001',
 'output_batch_10-02_miaSimS4_run01_5_40_th0_001',
 'output_batch_10-02_miaSimS4_run01_5_60_th0_001',
 'output_batch_10-02_miaSimS4_run02_10_100_th0_001',
 'output_batch_10-02_miaSimS4_run02_10_40_th0_001',
 'output_batch_10-02_miaSimS4_run02_10_60_th0_001',
 'output_batch_10-02_miaSimS4_run02_15_100_th0_001',
 'output_batch_10-02_miaSimS4_run02_15_40_th0_001',
 'output_batch_10-02_miaSimS4_run02_15_60_th0_001',
 'output_batch_10-02_miaSimS4_run02_5_100_th0_001',
 'output_batch_10-02_miaSimS4_run02_5_40_th0_001',
 'output_batch_10-02_miaSimS4_run02_5_60_th0_001',
 'output_batch_10-02_miaSimS4_run03_10_100_th0_001',
 'output_ba

### Create Plots directly from data

In [3]:
for hl_number in [5, 10, 15]: #[20, 50]:
    for hl_size in [40, 60, 100]: #, 80, 100
        for th in ["0_001"]: # "0_1", "0_01", 
            # get list of available runs for the given setting
            runs = []
            for file in [s for s in os.listdir(out_dir) if f'_{hl_number}_{hl_size}_th{th}' in s]:
                m = re.search('run(.+?)_', file)
                if m:
                    runs.append(m.group(1))

            # specify dimensions of the plot
            n_runs = len(runs)
            n_row = n_runs
            n_col = n_taxa

            # make plot
            fig, axs = plt.subplots(n_row, n_col)
            fig.set_figwidth(15)
            fig.set_figheight(3*n_runs)
            fig.suptitle(f"""{batch}\n
                        hidden layers: number={hl_number} of size={hl_size},\n
                        threshold = {th}""",
                        size=14, y=1.0)
            fig.tight_layout(pad=2.0)
            y = 0
            for run in runs:
                plot_path_tmp = f"{out_dir}output_{batch}_run{run}_{hl_number}_{hl_size}_th{th}/Data/"
                if os.path.exists(f"{plot_path_tmp}prediction_x1.csv"):
                    for taxon in np.arange(1,(n_taxa+1)):
                        # read data files
                        df_pred = pd.read_csv(f"{plot_path_tmp}prediction_x{taxon}.csv", header=[0])
                        df_data_train = pd.read_csv(f"{plot_path_tmp}data_train_x{taxon}.csv", header=[0])
                        # convert files to numpy array
                        data_train = np.array(df_data_train)
                        pred = np.array(df_pred)
                        # make plot
                        axs[int(y/n_col), (taxon-1)].plot(data_train[:,1], data_train[:,0], label = "data")
                        axs[int(y/n_col), (taxon-1)].plot(pred[:,1], pred[:,0], label = "u_hat")
                        axs[int(y/n_col), (taxon-1)].legend()
                        axs[int(y/n_col), (taxon-1)].set_title(f"x{taxon}")
                        # axs[int(y/n_col), (taxon-1)].title.set_size(10)
                        axs[int(y/n_col), (taxon-1)].set_ylim([-0.05, 0.5])
                        y += 1
                    axs[int((y-4)/n_col), 0].annotate(f"run {run}", xy=(0, 0.5), 
                                                    xytext=(-axs[int((y-4)/n_col), 0].yaxis.labelpad - 5, 0),
                            xycoords=axs[int((y-4)/n_col), 0].yaxis.label, textcoords='offset points',
                            size='large', ha='right', va='center')
            plt.savefig(f'{out_dir}/predictions_{batch}_{hl_number}_{hl_size}_th{th}.pdf',
                        bbox_inches='tight', dpi = 200)
            plt.close()